# Sentiment Analysis

Three ways to do sentiment analysis:
1. Lexicon/Keyword
2. NLP/ML
3. LLM/Gen-AI
<br>

Lexicon is outdated and NLP or LLM approach is more preferred today.

This notebook will compare the difference between these three method on sentiment analysis

## Dataset

We will be using sentiment140 dataset from https://www.kaggle.com/datasets/kazanova/sentiment140/data for sentiment analysis.

It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 2 = neutral, 4 = positive) and they can be used to detect sentiment .
It contains the following 6 fields:

1. target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
2. ids: The id of the tweet ( 2087)
3. date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
4. flag: The query (lyx). If there is no query, then this value is NO_QUERY.
5. user: the user that tweeted (robotickilldozr)

## Lexicon

Sentiment Analysis using lexicon works by:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [2]:
file_path = os.path.join('..', 'Data', 'training.1600000.processed.noemoticon.csv')
sentiment140_data = pd.read_csv(file_path, encoding='latin1', header=None)
sentiment140_data.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [3]:
sentiment140_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
sentiment140_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [5]:
sentiment140_data.describe(include=['object'])

,date,flag,user,text
count,1600000,1600000,1600000,1600000
unique,774363,1,659775,1581466
top,Mon Jun 15 12:53:14 PDT 2009,NO_QUERY,lost_dog,isPlayer Has Died! Sorry
freq,20,1600000,549,210


In [6]:
sentiment140_data['ids'].nunique()

1598315

In [7]:
# remove duplicates
sentiment140_data = sentiment140_data.drop_duplicates(subset=['ids'])
sentiment140_data.reset_index(drop=True, inplace=True)

# delete flag column
sentiment140_data = sentiment140_data.drop(['flag'], axis = 1)

In [8]:
sentiment140_data.head()

,target,ids,date,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
import nltk

## NLP/ML

## LLM/Gen-AI